In [1]:
import os
import pickle

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from googleapiclient.discovery import build

In [2]:
credentials = None

# tokens.pickle stores the user's credentials from previously successful logins
if os.path.exists("token.pickle"):
    print("Loading Credentials From File...")
    # accessing the credentials from the pickle file
    with open("token.pickle", "rb") as token:
        credentials = pickle.load(token)

# if there is no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print("Refreshing Acess Token...")
        credentials.refresh(Request())
    else:
        print("Fetching New Tokens...")
        flow = InstalledAppFlow.from_client_secrets_file(
                            "client_secrets.json", # Run the O2auth authorisation code 
                            scopes=["https://www.googleapis.com/auth/youtube.force-ssl"] #determine the scope till what you want to access
                            )
        # start running the server to collect the authorisation credentials 
        flow.run_local_server(
                        port=8080, # port on which the code should run
                        prompt="consent",  # the message which should be prompt before the user authorizing the access
                        authorization_prompt_message = ""
                        )
        #store the credentials into a variable named credentials
        credentials = flow.credentials
        # dump the credentials back into the pickle file
        with open("token.pickle", "wb") as f:
            print("Saving Credentials for Fututre Use...")
            pickle.dump(credentials, f)

Fetching New Tokens...

Saving Credentials for Fututre Use...


<b>Acessing The Youtube Account using Credentials</b>

In [3]:
youtube = build("youtube", "v3",  credentials=credentials)

<b>Acessing comments posted on the Video till now</b>

In [4]:
request = youtube.commentThreads().list(
     part="id, replies, snippet",  videoId="8JlcXr11084"
)
response = request.execute()

# print(response)

# printing comments posted on the video
for snip in response['items']:
    print("Comment Id:" , snip['id'] , end="    ")
    print("Comment: ", snip['snippet']['topLevelComment']['snippet']['textDisplay'])

    if snip.get('replies', None):
        for rep in snip['replies']['comments']:
            print("Reply Comment Id: " , rep['id'], end="   ")
            print("Comment: ", rep['snippet']['textDisplay'])



Comment Id: UgwHGt4c3W2VPWqlLB54AaABAg    Comment:  Yeh black screen lagane ka idea kaha se aaya.
Comment Id: UgwF_nnUt-kjlX0-fRd4AaABAg    Comment:  Tum jaise logo ko video nhi banai chahiyeh
Comment Id: UgyER8tNiWYZJ9zrIr54AaABAg    Comment:  Poora mood kharab kr diya. Kya bekar video hain
Comment Id: UgzTatzjK5dp3yCnxSp4AaABAg    Comment:  All the best aakash😊. We are waiting for your upcoming videos.


<b>Removing a Comment using --- setModerationStatus</b><br> <b>setModerationStatus</b> provides us 4 options to moderate the comment, which are <br> <b>a. published <br> b. heldForReview <br> c. likelySpam <br> d. rejected</b>

In [5]:
# # removing comment written by others but 
# # not removing comments written by the creator himself
# request = youtube.comments().setModerationStatus(
#         id="UgzAxTGTvYvtfCQ2Btl4AaABAg",
#         moderationStatus="published"
#     )
# request.execute()

<b>Output Structure of "response"</b>

In [6]:
# {
#     'kind': 'youtube#commentThreadListResponse', 
#     'etag': 'tt_RxG-7XES4qKO4gewIRRbPHcY', 
#     'pageInfo': {'totalResults': 9, 'resultsPerPage': 20}, 
#     'items': [
#             {'kind': 'youtube#commentThread', 
#              'etag': 'oWUWsLK9wc6TWdEtMQnbavtyG3g', 
#              'id': 'UgxPf-VFkiqx3VRBqll4AaABAg', 
#              'snippet': {
#                         'videoId': '8JlcXr11084', 
#                         'topLevelComment': {
#                                             'kind': 'youtube#comment', 
#                                             'etag': 'k80167Mt66S_Ssa_48_kYiI7FWo', 
#                                             'id': 'UgxPf-VFkiqx3VRBqll4AaABAg', 
#                                             'snippet': {
#                                                         'videoId': '8JlcXr11084', 
#                                                         'textDisplay': 'Today is very lazy day😢', 
#                                                         'textOriginal': 'Today is very lazy day😢', 
#                                                         'authorDisplayName': 'Fabsurbhitcoding', 
#                                                         'authorProfileImageUrl': 'https://yt3.ggpht.com/bVGapP8wW4WJzLlLEeCOzJ-9SakBVqm7VZ6z4QK1e-3Bm8Eq7481oHQqAzUEahmvC69Vx4E-sl4=s48-c-k-c0x00ffffff-no-rj', 
#                                                         'authorChannelUrl': 'http://www.youtube.com/channel/UC1-B4kBKMdrGIQaNe1iloRQ', 
#                                                         'authorChannelId': {'value': 'UC1-B4kBKMdrGIQaNe1iloRQ'}, 
#                                                         'canRate': True, 
#                                                         'viewerRating': 'none', 
#                                                         'likeCount': 0, 'publishedAt': 
#                                                         '2023-04-13T13:16:50Z', 
#                                                         'updatedAt': '2023-04-13T13:16:50Z'
#                                                         }
#                                             }, 
#                         'canReply': True, 
#                         'totalReplyCount': 0, 
#                         'isPublic': True
#                         }
#             }, 
#     ]
# }

<b>Replying to a Top-Level-Comment in a Video</b>

In [7]:
# request = youtube.comments().insert(
#         part="snippet",
#         body={
#           "snippet": {
#             "parentId": "Ugx-4B6X1yVh3wstW214AaABAg",
#             "textOriginal": "One More reply."
#           }
#         }
#     )

# response = request.execute()
# print("Parent Id: ", response['snippet']['parentId'])
# print("Comment Id: " , response['id'], end="    ")
# print("Comment: ", response['snippet']['textDisplay'])

<b>Inserting a Top-Level-Comment in a Video</b>

In [8]:
# request = youtube.commentThreads().insert(
#         part="snippet",
#         body={
#           "snippet": {
#             "videoId": "8JlcXr11084",
#             "topLevelComment": {
#               "snippet": {
#                 "textOriginal": "Nanga nachega tu"
#               }
#             }
#           }
#         }
#       )

# response = request.execute()
# print("Comment Id: " , response['id'], end="    ")
# print("Comment: ", response['snippet']['topLevelComment']['snippet']['textDisplay'])

In [9]:
# from googleapiclient.errors import HttpError
# import time

<b>Updating a Comment --- method not working </b>

In [10]:
# try:
#   request = youtube.comments().update(
#         part="id, snippet",
#         body={
#           "id": "Ugwb0y3OEPf7f5RMHzd4AaABAg",
#           "snippet": {
#             "textOriginal": "Updated comment. Hurray!"
#           }
#         }
#     )
#   request.execute()
# except HttpError as err:
#     print(err.resp.status)
#     print(err.reason)

# print("Comment Id: " , response['id'], end="    ")
# print("Comment: ", response['snippet']['textDisplay'])

<b>Deleting a Comment --- method not working</b>

In [11]:
# try:
#     request = youtube.comments().delete(
#         id="Ugwb0y3OEPf7f5RMHzd4AaABAg",
#     )
#     request.execute()
# except HttpError as err:
#     print(err.resp.status)
#     print(err.reason)